In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
X,y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=2,
    n_redundant=8,
    flip_y=0,
    weights=[0.9,0.1],
    random_state=42
)
np.unique(y,return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=42)

## Experiment 1: Train Logistic Regression Model

In [4]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



## Experiment 2: Train Random Forest Classifier

In [5]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.95      0.70      0.81        30

    accuracy                           0.97       300
   macro avg       0.96      0.85      0.89       300
weighted avg       0.97      0.97      0.96       300



## Experiment 3: Train XGBoost

In [6]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



## Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [7]:
np.unique(y_train,return_counts=True)

(array([0, 1]), array([630,  70], dtype=int64))

In [8]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_smt,y_train_smt = smt.fit_resample(X_train,y_train)
np.unique(y_train_smt,return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

In [9]:
y_train.shape

(700,)

In [10]:
y_train_smt.shape

(1238,)

In [11]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_smt, y_train_smt)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



### Track Experiments using ML Flow

In [12]:
models = [
    (
        "Logistic Regression",
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(C=1,solver="liblinear"),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(n_estimators=30, max_depth=3), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
        (X_train_smt, y_train_smt),
        (X_test, y_test)
    ),
]

In [13]:
import mlflow

In [26]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("Anomaly_Detection")

for model_name,params,model,train,test in models:
    X_train,y_train = train
    X_test,y_test = test

    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test,y_pred,output_dict=True)

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params({"params":params,"model":model_name})
        mlflow.log_metrics({
            'accuracy':report['accuracy'],
            'recall_class_0':report['0']['recall'],
            'recall_class_1':report['1']['recall'],
            'f1_score_macro':report['macro avg']['f1-score']
        })

        if 'XGB' in model_name:
            mlflow.xgboost.log_model(model,f"{model_name}")
        else:
            mlflow.sklearn.log_model(model,f"{model_name}")
            

2025/08/09 10:42:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/09 10:42:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/09 10:42:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://127.0.0.1:5000/#/experiments/232178928609931995/runs/2fb6a1f54f34486e8fdd44931d6437c7
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/232178928609931995


2025/08/09 10:42:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/09 10:42:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/232178928609931995/runs/094d4466c9fa4d029e9c479a5ff27d03
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/232178928609931995


2025/08/09 10:42:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/08/09 10:42:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://127.0.0.1:5000/#/experiments/232178928609931995/runs/1c9bf757ca0c40b59145c48f8a8cc016
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/232178928609931995


2025/08/09 10:42:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://127.0.0.1:5000/#/experiments/232178928609931995/runs/b9c32544d2034adeaf34ef1ae5b769c5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/232178928609931995


## Register the Model

In [29]:
model_name = "XGBClassifier With SMOTE"
run_id = input("Enter model run id: ")
model_uri = f"runs:/{run_id}/{model_name}"

result = mlflow.register_model(model_uri,model_name)

Enter model run id:  b9c32544d2034adeaf34ef1ae5b769c5


Successfully registered model 'XGBClassifier With SMOTE'.
2025/08/09 10:47:46 WARNING mlflow.tracking._model_registry.fluent: Run with id b9c32544d2034adeaf34ef1ae5b769c5 has no artifacts at artifact path 'XGBClassifier With SMOTE', registering model based on models:/m-40cd1386c0d1487a8518b8a2a9ca5161 instead
2025/08/09 10:47:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBClassifier With SMOTE, version 1
Created version '1' of model 'XGBClassifier With SMOTE'.


## Load the Model

In [36]:
model_version = 1
model = mlflow.xgboost.load_model(f"models:/{model_name}/{model_version}")
y_pred = model.predict(X_test)
y_pred[:5]

array([0, 0, 0, 0, 0])

In [38]:
# another way of loading the model
model_version = 1
model = mlflow.xgboost.load_model(f"models:/{model_name}@challenger")
y_pred = model.predict(X_test)
y_pred[:5]

array([0, 0, 0, 0, 0])

In [39]:
dev_model_uri = f"models:/{model_name}@challenger"
prod_model = 'anomaly-detection-prod'
client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=dev_model_uri,dst_name=prod_model)

Successfully registered model 'anomaly-detection-prod'.
Copied version '1' of model 'XGBClassifier With SMOTE' to version '1' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1754719295973, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1754719295973, metrics=None, model_id=None, name='anomaly-detection-prod', params=None, run_id='b9c32544d2034adeaf34ef1ae5b769c5', run_link='', source='models:/XGBClassifier With SMOTE/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

## Load Production Model

In [40]:
model = mlflow.xgboost.load_model(f"models:/{prod_model}@champion")
y_pred = model.predict(X_test)
y_pred[:5]

array([0, 0, 0, 0, 0])